In [29]:
import cv2
import matplotlib
import numpy as np
import os
from matplotlib import pyplot as plt
# Import interaction widgets and functions
from ipywidgets import interact, interactive, interact_manual

def my_threshold(thresh, modeStr):
    global change
    global dst
    mode = eval(modeStr)
    #th, dst = cv2.threshold(change,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    th, dst = cv2.threshold(change, thresh, 255, mode)
    


# Load the image in color (OpenCV standard = BGR)
ref = cv2.imread(r"C:\Users\camba\Data_science\cogum_images\zero.jpeg", cv2.IMREAD_COLOR)
# Convert to RGB
#orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

# Load the image in grayscale
#gray = cv2.imread("/content/drive/MyDrive/vision/data/cogum/min2.jpeg", cv2.IMREAD_GRAYSCALE)

#blur = cv2.GaussianBlur(gray,(7,7),6)

lab= cv2.cvtColor(ref, cv2.COLOR_BGR2LAB)
l_channel, a, b = cv2.split(lab)

# Applying CLAHE to L-channel
# feel free to try different values for the limit and grid size:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl = clahe.apply(l_channel)

# merge the CLAHE enhanced L-channel with the a and b channel
limg = cv2.merge((cl,a,b))

# Converting image from LAB Color model to BGR color spcae
enhanced_img_ref = cv2.cvtColor(limg, cv2.COLOR_BGR2GRAY)

directory = r"C:\Users\camba\Data_science\cogum_images\video_raw"
large_area = 10000
number = 1
for filename in os.listdir(directory):
    # Check if the file is a JPEG image
    if filename.endswith('.jpeg'):
        # Define the filepath for the current image
        filepath = os.path.join(directory, filename)
        
        # Load the image
        orig = cv2.imread(filepath, cv2.IMREAD_COLOR)
        
        # Check if the image was loaded successfully
        if orig is None:
            print(f"Error loading image: {filepath}")
            continue
        
        # Convert the color space from BGR to LAB
        lab= cv2.cvtColor(orig, cv2.COLOR_BGR2LAB)
        l_channel, a, b = cv2.split(lab)

        # Applying CLAHE to L-channel
        # feel free to try different values for the limit and grid size:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        cl = clahe.apply(l_channel)

        # merge the CLAHE enhanced L-channel with the a and b channel
        limg = cv2.merge((cl,a,b))

        # Converting image from LAB Color model to BGR color space
        enhanced_img = cv2.cvtColor(limg, cv2.COLOR_BGR2GRAY)

        change = cv2.absdiff(enhanced_img_ref, enhanced_img)

        # Thresholding
        th, dst = cv2.threshold(change,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        # Multiplying
        masked = cv2.multiply(dst, (dst*6))

        # Morphological operations
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
        opening = cv2.morphologyEx(dst, cv2.MORPH_OPEN, kernel)

        kernel2 = np.ones((19,19),np.uint8)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel2)

        # Find contours
        contours, _ = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Iterate through contours
        
        for i, contour in enumerate(contours):
            # Filter out contours based on area
            area = cv2.contourArea(contour)
            if number<=17:
                if 4000 < area < (large_area):  # Define your threshold here
                    print("found", area)
                    number=number+1
                    # Draw contour lines on the original image
                    cv2.drawContours(orig, [contour], -1, (100, 0, 0), 3)
                    # Annotate contour with its index
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cX = int(M["m10"] / M["m00"])+30
                        cY = int(M["m01"] / M["m00"])
                        cv2.putText(orig, str(area), (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            else:
                if 4000 < area < (large_area*2):  # Define your threshold here
                    print("found", area)
                    number=number+1
                    # Draw contour lines on the original image
                    cv2.drawContours(orig, [contour], -1, (100, 0, 0), 3)
                    # Annotate contour with its index
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cX = int(M["m10"] / M["m00"])+30
                        cY = int(M["m01"] / M["m00"])
                        cv2.putText(orig, str(area), (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        large_area = large_area +2000
        
        multiplier = multiplier *1.1
        output_filepath = os.path.join(directory, f"modified_{filename}")
        cv2.imwrite(output_filepath, orig)

found 6352.5
found 7814.5
found 5366.5
found 4420.5
found 8479.5
found 9099.0
found 11471.0
found 14554.5
found 15583.5
found 18292.5
found 19646.5
found 18995.5
found 26496.0
found 29385.5
found 16648.5
found 33123.5
found 34251.5
found 64513.0
found 36103.0
found 85372.5
found 43976.0
found 4241.0
found 91949.5
found 57521.0
found 6655.0
found 4127.5
found 8468.5
found 63635.0
found 4682.0
found 14080.5
found 4039.0
found 92141.0
found 69369.0
found 5075.0
found 14956.5
found 7198.5
found 96204.0
found 69331.5
found 15039.0
found 5378.0
found 115449.5
found 73305.5
found 14384.0
found 5906.5
found 93526.0
found 80121.0
found 15467.0
found 6424.0
found 78867.5
found 74842.0
found 10437.0
found 4438.5
found 6660.5
found 83533.0
found 77101.5
found 7568.0
found 7047.5
found 122291.5
found 8182.0
found 94498.5
found 19516.5
found 8285.0
found 14989.5
found 43620.0
found 75800.0
found 7585.5
found 8041.5
found 79448.5
found 4003.5
found 79444.0
found 12210.0
found 5195.5
found 9301.0
foun

In [5]:
video_dir = os.path.join('../cogum_images/video')
